In [1]:
import sys 

In [2]:
cd '/srv/share3/hagrawal9/project/habitat/habitat-api/'

/coc/pskynet3/hagrawal9/project/habitat/habitat-api


In [36]:
import gzip
import json
import os
import sys
import itertools
from typing import Any, Dict, List, Optional, Type
from copy import deepcopy

import attr
import cv2
import git
import magnum as mn
import numpy as np

# %matplotlib inline
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image


import habitat
import habitat_sim
from habitat.config import Config
from habitat.core.registry import registry
from habitat_sim.utils import viz_utils as vut
from habitat.utils.visualizations import maps, fog_of_war
from habitat.utils.visualizations.utils import observations_to_image

from rearrangement.utils.planner import (
    compute_traversable_map,
    compute_distance_using_fmm,
    find_dist_from_map,
    compute_distance_mat_using_navmesh,
    compute_distance_mat_using_fmm,
    find_shortest_path_for_multiple_objects
)
from rearrangement.utils.visualization import (
    get_top_down_map
)
from rearrangement.utils.geometry import (
     geodesic_distance
)
from collections import defaultdict

In [4]:
from habitat.datasets.rearrangement.rearrangement_dataset import RearrangementDatasetV0
from habitat.tasks.rearrangement.rearrangement_task import RearrangementEpisode

In [5]:
repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
# %cd $dir_path
data_path = os.path.join(dir_path, "data")
output_directory = "data/tutorials/output/"  # @param {type:"string"}
output_path = os.path.join(dir_path, output_directory)

In [51]:
config = habitat.get_config("configs/tasks/rearrangement_gibson.yaml")

In [56]:
config.defrost()
config.TASK.MEASUREMENTS = ['OBJECT_TO_GOAL_DISTANCE', 'AGENT_TO_OBJECT_DISTANCE', 'TOP_DOWN_MAP']
config.DATASET.SPLIT = "val"
config.freeze()

In [57]:
config.DATASET

Config({'TYPE': 'RearrangementDataset-v0', 'SPLIT': 'val', 'SCENES_DIR': 'data/scene_datasets', 'CONTENT_SCENES': ['*'], 'DATA_PATH': 'data/datasets/rearrangement/gibson/v1/{split}/{split}.json.gz'})

In [58]:
config.TASK.TOP_DOWN_MAP.MAP_RESOLUTION

256

In [59]:
try:
    env.close()
except:
    pass
env = habitat.Env(config)
fog_of_war_mask = None
pickup_order = {}


2020-10-27 11:40:42,840 Initializing dataset RearrangementDataset-v0
2020-10-27 11:40:43,126 initializing sim RearrangementSim-v0
I1027 11:40:48.643167 33504 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Hallettsville.navmesh
I1027 11:40:48.644935 33504 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.
2020-10-27 11:40:48,921 Initializing task RearrangementTask-v0


In [60]:
len(env.episodes)

1000

In [93]:
env.current_episode.start_position

[-11.284797668457031, 0.17394161224365234, -5.5449724197387695]

In [61]:
def start_env_episode_distance(env, pickup_order):
    
    traversal_order = []
    pathfinder = env._task._simple_pathfinder
    episode = env.current_episode 
    # dist_mat = compute_distance_mat_using_navmesh(env._task._simple_pathfinder, agent_pos, object_positions, goal_positions)
    
    agent_pos = env._sim.get_agent(0).get_state().position
    object_positions = [obj.position for obj in episode.objects]
    goal_positions = [obj.position for obj in episode.goals]
    
    prev_obj_end_pos = agent_pos
    shortest_dist = 0

    for i in range(len(pickup_order)):
        curr_idx = pickup_order[i] - 1
        curr_obj_start_pos = object_positions[curr_idx]
        curr_obj_end_pos = goal_positions[curr_idx]
        shortest_dist += geodesic_distance(
                pathfinder, prev_obj_end_pos, [curr_obj_start_pos]
            )

        shortest_dist += geodesic_distance(
                    pathfinder, curr_obj_start_pos, [curr_obj_end_pos]
                ) 
        prev_obj_end_pos = curr_obj_end_pos
        
        
    return {
        'episode_id': episode.episode_id, 
        'scene_id': episode.scene_id,
        'episode_dist': shortest_dist
    }

In [91]:
obs = env.reset()
all_permutations = list(itertools.permutations(range(1, len(env.current_episode.objects) + 1)))
min_dist = 1000
for pickup_order in all_permutations:
    is_oracle = False
    d = start_env_episode_distance(env, pickup_order)
    if list(pickup_order) == list(env.current_episode.pickup_order):
        is_oracle = True
        oracle_dist = d['episode_dist']
    if d['episode_dist'] < min_dist:
        min_dist = d['episode_dist']
        min_order = pickup_order
        
    # print("Oracle: {}".format(is_oracle), pickup_order, d['episode_dist'])
if min_dist < oracle_dist:
    print("found shorter path")

print(min_dist, min_order)
print(oracle_dist, env.current_episode.pickup_order)


39.98417782783508 (1, 2)
39.98417782783508 [1, 2]


In [ ]:
class SPL(Measure):
    r"""SPL (Success weighted by Path Length)
    """

    def __init__(
        self, sim, config: Config, *args: Any, **kwargs: Any
    ):
        self._previous_position = None
        self._start_end_episode_distance = None
        self._agent_episode_distance = None
        self._episode_view_points = None
        self._sim = sim
        self._config = config

        super().__init__()

    def _get_uuid(self, *args: Any, **kwargs: Any) -> str:
        return "spl"

    def reset_metric(self, episode, task, *args: Any, **kwargs: Any):
        task.measurements.check_measure_dependencies(
            self.uuid, [DistanceToGoal.cls_uuid, Success.cls_uuid]
        )

        self._previous_position = self._sim.get_agent_state().position
        self._agent_episode_distance = 0.0
        self._start_end_episode_distance = task.measurements.measures[
            DistanceToGoal.cls_uuid
        ].get_metric()
        self.update_metric(episode=episode, task=task, *args, **kwargs)

    def _euclidean_distance(self, position_a, position_b):
        return np.linalg.norm(position_b - position_a, ord=2)

    def update_metric(
        self, episode, task: EmbodiedTask, *args: Any, **kwargs: Any
    ):
        ep_success = task.measurements.measures[Success.cls_uuid].get_metric()

        current_position = self._sim.get_agent_state().position
        self._agent_episode_distance += self._euclidean_distance(
            current_position, self._previous_position
        )

        self._previous_position = current_position

        self._metric = ep_success * (
            self._start_end_episode_distance
            / max(
                self._start_end_episode_distance, self._agent_episode_distance
            )
        )

In [44]:
# # Scratch Code
# for objid in env._sim.get_existing_object_ids():
#     if objid != env._sim.agent_object_id:
#         episode_obj_id = env._task.sim_object_to_objid_mapping[objid]
#         dist = geodesic_distance(env._sim.pathfinder, agent_pos, env.current_episode.objects[episode_obj_id].position)
#         if dist == np.inf:
#             print("Voila!!")


In [10]:
len(env.episodes)

35000

In [48]:
while(len(pickup_order)!=len(env.episodes)):
    data = compute_l2dist_pickup_order(env)
    pickup_order[data['episode_id'] + '_' + data['scene_id']] = data
    print('\rEpisode ID: {} Done: {} '.format(data['episode_id'], len(pickup_order)), end=" ")

Episode ID: 19 Done: 1000  

In [49]:
with open('data/pickup_order_l2dist_test.json','w') as f:
    json.dump(pickup_order, f)

In [62]:
with open('data/pickup_order_l2dist_train.json','r') as f:
    pickup_order_l2dist = json.load(f)
    
with open('data/pickup_order_tdmap_train.json','r') as f:
    pickup_order_tdmap = json.load(f)

In [63]:
updated_scene_data = []
split = 'train'

for filename in os.listdir('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split)):
    scene_id = filename.split('.')[0].split('_')[-1]
    with gzip.open('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split) + filename, "rt") as f:
        scene_data = json.load(f)
    updated_scene_data = deepcopy(scene_data)
    
    for i, episode in enumerate(scene_data['episodes']):
        episode_id = episode['episode_id']
        assert updated_scene_data['episodes'][i]['episode_id'] == episode['episode_id']
        assert updated_scene_data['episodes'][i]['scene_id'] == episode['scene_id']
        updated_scene_data['episodes'][i]['pickup_order_tdmap'] = pickup_order_tdmap[str(episode['episode_id']) + "_" + episode['scene_id']]['pickup_order_fmm']
        updated_scene_data['episodes'][i]['pickup_order_l2dist'] = pickup_order_l2dist[str(episode['episode_id']) + "_" + episode['scene_id']]['pickup_order_l2dist']
    
    
    with gzip.open('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split) + filename, "wt") as f:
        json.dump(updated_scene_data, f)
        
    print(filename)

rearrangement_v1_train_n=1000_o=5_Monson.json.gz
rearrangement_v1_train_n=1000_o=5_Scandinavia.json.gz
rearrangement_v1_train_n=1000_o=5_Hobson.json.gz
rearrangement_v1_train_n=1000_o=5_Barboursville.json.gz
rearrangement_v1_train_n=1000_o=5_Mentasta.json.gz
rearrangement_v1_train_n=1000_o=5_Avonia.json.gz
rearrangement_v1_train_n=1000_o=5_Ewell.json.gz
rearrangement_v1_train_n=1000_o=5_Gilbert.json.gz
rearrangement_v1_train_n=1000_o=5_Sawpit.json.gz
rearrangement_v1_train_n=1000_o=5_Montreal.json.gz
rearrangement_v1_train_n=1000_o=5_Orason.json.gz
rearrangement_v1_train_n=1000_o=5_Hainesburg.json.gz
rearrangement_v1_train_n=1000_o=5_Helix.json.gz
rearrangement_v1_train_n=1000_o=5_Westerville.json.gz
rearrangement_v1_train_n=1000_o=5_Arona.json.gz
rearrangement_v1_train_n=1000_o=5_Delton.json.gz
rearrangement_v1_train_n=1000_o=5_Roeville.json.gz
rearrangement_v1_train_n=1000_o=5_Euharlee.json.gz
rearrangement_v1_train_n=1000_o=5_Hominy.json.gz
rearrangement_v1_train_n=1000_o=5_Bowlus.j